In [27]:
import psycopg2
import csv
import glob
import numpy as np 
from pandasql import sqldf
import pandas as pd


In [28]:
pysqldf = lambda q: sqldf(q, globals())

In [6]:
#create the database

# ! createdb mod2project


In [2]:
#connect to the database and make a cursor
conn = psycopg2.connect("dbname=mod2project")
cur = conn.cursor()

In [4]:
#set up our queries
create_query_1 = """
CREATE TABLE course_offerings(
  uuid text
, course_uuid text 
, term_code integer
, name text
)
"""

create_query_2 = """
CREATE TABLE courses(
  uuid text 
, name text 
, number integer
)
"""

create_query_3 = """
CREATE TABLE grade_distributions(
  course_offering_uuid text
, section_number text 
, a_count integer
, ab_count integer
, b_count integer
, bc_count integer
, c_count integer
, d_count integer
, f_count integer
, s_count integer
, u_count integer
, cr_count integer
, n_count integer
, p_count integer
, i_count integer
, nw_count integer
, nr_count integer
, other_count integer
)
"""

create_query_4 = """
CREATE TABLE instructors(
  id integer
, name text
)
"""

create_query_5 = """
CREATE TABLE rooms(
  uuid text
, facility_code text
, room_code text
)
"""

create_query_6 ="""
CREATE TABLE schedules(
  uuid text
, start_time integer
, end_time integer
, mon text
, tues text
, wed text
, thurs text
, fri text
, sat text
, sun text
)
"""

create_query_7 = """
CREATE TABLE sections(
  uuid text
, course_offering_uuid text
, section_type text
, number text
, room_uuid text
,schedule_uuid text
)
"""

create_query_8 = """
CREATE TABLE subject_memberships(
  subject_code integer
, course_offering_uuid text
)
"""

create_query_9 = """
CREATE TABLE subjects(
  code text
, name text
, abbreviation text
)
"""

create_query_10 = """
CREATE TABLE teachings(
  instructor_id integer
, section_uuid text
)
"""

In [5]:
#get the names and locations of our datafiles
csvs_for_table = glob.glob('raw_data/*.csv')

In [6]:
#make a list of our queries
queries_list = [create_query_1,create_query_2,create_query_3,create_query_4
               ,create_query_5,create_query_6,create_query_7
               ,create_query_8,create_query_9,create_query_10]

In [7]:
#extract table names from our file names
table_names = [x.split('/')[1][:-4] for x in csvs_for_table]

In [8]:
#loop through our queries list and execute them

for query in queries_list:
    cur.execute(query)
    conn.commit()

In [9]:
#loop through our table names and indexes
for q_num, table_name in enumerate(table_names):
    
    #extract values from our first queries to use
    #in our insert statements below
    fields = tuple(line.strip(", ").split()[0] for 
                   line in queries_list[q_num].split('\n')[2:-2])
    
    #set up our insert statements
    query = f'''
    INSERT INTO {table_name}
    ({', '.join(fields)})
    VALUES ({', '.join('%s' for _ in range(len(fields)))})
    '''
        
    
    #open each file and insert it's values row
    #by row into the appropriate tables
    with open(csvs_for_table[q_num], 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row.
        for row in reader:
            cur.execute(
                query,
                row
            )
            conn.commit()
            
        #check where we're having problems
        print(table_name,'still good')

course_offerings still good
courses still good
grade_distributions still good
instructors still good
rooms still good
schedules still good
sections still good
subject_memberships still good
subjects still good
teachings still good


In [23]:
conn.rollback()


In [36]:
import sqlalchemy
from sqlalchemy import create_engine


engine = create_engine("postgresql:///mod2project", echo=True)


In [126]:
#this was a pain in many places
q = """

SELECT *,
    CASE
        WHEN term_code BETWEEN 1063 AND 1073 THEN '2006'
        WHEN term_code BETWEEN 1073 AND 1083 THEN '2007'
        WHEN term_code BETWEEN 1083 AND 1093 THEN '2008'
        WHEN term_code BETWEEN 1093 AND 1103 THEN '2009'
        WHEN term_code BETWEEN 1103 AND 1113 THEN '2010'
        WHEN term_code BETWEEN 1113 AND 1123 THEN '2011'
        WHEN term_code BETWEEN 1123 AND 1133 THEN '2012'
        WHEN term_code BETWEEN 1133 AND 1143 THEN '2013'
        WHEN term_code BETWEEN 1143 AND 1153 THEN '2014'
        WHEN term_code BETWEEN 1153 AND 1163 THEN '2015'
        WHEN term_code BETWEEN 1163 AND 1173 THEN '2016'
        WHEN term_code BETWEEN 1173 AND 1183 THEN '2017'
        ELSE 'the_world_is_over'
    END AS Year
    FROM course_offerings;
"""


In [124]:
df = pd.read_sql_query(q,con=engine)

2019-05-07 14:31:44,793 INFO sqlalchemy.engine.base.Engine 

SELECT *,
    CASE
        WHEN term_code BETWEEN 1063 AND 1073 THEN '2006'
        WHEN term_code BETWEEN 1073 AND 1083 THEN '2007'
        WHEN term_code BETWEEN 1083 AND 1093 THEN '2008'
        WHEN term_code BETWEEN 1093 AND 1103 THEN '2009'
        WHEN term_code BETWEEN 1103 AND 1113 THEN '2010'
        WHEN term_code BETWEEN 1113 AND 1123 THEN '2011'
        WHEN term_code BETWEEN 1123 AND 1133 THEN '2012'
        WHEN term_code BETWEEN 1133 AND 1143 THEN '2013'
        WHEN term_code BETWEEN 1143 AND 1153 THEN '2014'
        WHEN term_code BETWEEN 1153 AND 1163 THEN '2015'
        WHEN term_code BETWEEN 1163 AND 1173 THEN '2016'
        WHEN term_code BETWEEN 1173 AND 1183 THEN '2017'
        ELSE 'the_world_is_over'
    END AS Year
    FROM course_offerings;

2019-05-07 14:31:44,794 INFO sqlalchemy.engine.base.Engine {}


In [125]:
df

,uuid,course_uuid,term_code,name,year
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog,2008
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog,2007
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog,2016
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog,2011
4,2b4e216d-a728-3713-8c7c-19afffc6b2fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1104,Cooperative Education Prog,2010
5,87966a7b-f676-33d0-83d2-acdb67da6790,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1112,Cooperative Education Prog,2010
6,ff511882-5eab-3e7b-a89f-8fbfd1906127,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1134,Cooperative Education Prog,2013
7,b3e26604-fe6e-30df-bc91-b5fa57717a7e,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1084,Cooperative Education Prog,2008
8,e100d196-5e82-32e4-80e9-ac45c07a498c,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1154,Cooperative Education Prog,2015
9,76b9c458-d3c2-38c4-951f-69b6900bd7fe,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1162,Cooperative Education Prog,2015
